In [1]:
import pandas as pd
import numpy as np

In [17]:
import pickle

In [25]:
# Load pre-saved similarity matrices and location data
with open("location_distance.pkl", "rb") as file:
    location_df = pickle.load(file)
with open("cosine_sim1.pkl", "rb") as file:
    cosine_sim1 = pickle.load(file)
with open("cosine_sim2.pkl", "rb") as file:
    cosine_sim2 = pickle.load(file)
with open("cosine_sim3.pkl", "rb") as file:
    cosine_sim3 = pickle.load(file)

In [ ]:
def recommend_properties_with_scores(property_name, top_n=5):
    """Recommends properties based on similarity scores."""
    cosine_sim_matrix = 0.5 * cosine_sim1 + 0.8 * cosine_sim2 + 1 * cosine_sim3


    try:
        # Get the similarity scores for the given property
        sim_scores = list(enumerate(cosine_sim_matrix[location_df.index.get_loc(property_name)]))

        # Sort properties based on similarity scores
        sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the indices and scores of the top_n most similar properties
        top_indices = [i[0] for i in sorted_scores[1:top_n + 1]]
        top_scores = [i[1] for i in sorted_scores[1:top_n + 1]]

        # Retrieve the names of the top properties using the indices
        top_properties = location_df.index[top_indices].tolist()
        distance = location_df

        # Create a dataframe with the results
        recommendations_df = pd.DataFrame({
            'PropertyName': top_properties,
            'SimilarityScore': top_scores
            # 'Distance' : distance

        })

        return recommendations_df

    except KeyError:
        print(f"Error: '{property_name}' not found in location data.")
        return None


# recommendation_df = recommend_properties_with_scores('DLF The Camellias')
# print(recommendation_df)


In [69]:
(location_df.columns).shape

(1070,)

In [72]:
# User Interaction (CLI-based)
def main():
    
    print("\n==== Select Location and Radius ====")
    selected_location = input("Enter a location from the list:\n" + ", ".join(sorted(location_df.columns.to_list())) + "\n")

    try:
        radius = float(input("Enter radius in kilometers: "))
        result_ser = location_df[location_df[selected_location] < radius * 1000][selected_location].sort_values()

        print("\nProperties within", radius, "km:")
    
    
        for key, value in result_ser.items():
            
            print(f"{key}: {round(value / 1000)} kms")
        
    except KeyError:
        print("Invalid location selected.")
    except ValueError:
        print("Invalid input. Please enter a number for radius.")

    print("\n==== Recommend Apartments ====")
    selected_apartment = input("Enter an apartment name from the list:\n" + ", ".join(sorted(location_df.index.to_list())) + "\n")

    recommendation_df = recommend_properties_with_scores(selected_apartment)
    
    if recommendation_df is not None:
        print("\nRecommended Apartments:")
        print(recommendation_df.to_string(index=False))

if __name__ == "__main__":
    main()



==== Select Location and Radius ====

Properties within 5.0 km:

==== Recommend Apartments ====

Recommended Apartments:
             PropertyName  SimilarityScore
     Emaar MGF Palm Hills         0.765450
             M3M Latitude         0.720841
Orris Aster Court Premier         0.717884
              Trump Tower         0.637444
   Krrish Florence Estate         0.625059


Recommended Apartments:
PropertyName SimilarityScore
Emaar MGF Palm Hills 0.765450
M3M Latitude 0.720841
Orris Aster Court Premier 0.717884
Trump Tower 0.637444
Krrish Florence Estate 0.625059


In [83]:
import pandas as pd

def recommend_properties_with_scores(property_name, filtered_df, top_n=5):
    """Recommends properties based on similarity scores from the filtered list."""
    cosine_sim_matrix = 0.5 * cosine_sim1 + 0.8 * cosine_sim2 + 1 * cosine_sim3  # Adjusted weights

    try:
        # Get similarity scores for the selected property
        property_index = location_df.index.get_loc(property_name)
        sim_scores = list(enumerate(cosine_sim_matrix[property_index]))

        # Sort properties by similarity score (descending)
        sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get top_n most similar properties
        top_indices = [i[0] for i in sorted_scores[1:top_n + 1]]
        top_scores = [i[1] for i in sorted_scores[1:top_n + 1]]

        # Retrieve property names from indices
        top_properties = location_df.index[top_indices].tolist()

        # **Extract only properties within the selected radius**
        filtered_recommendations = [
            (prop, score, round(filtered_df.loc[prop].values[0] / 1000, 2))  # Extract distance
            for prop, score in zip(top_properties, top_scores)
            if prop in filtered_df.index
        ]

        # **Create DataFrame with recommendations**
        recommendations_df = pd.DataFrame(filtered_recommendations, columns=['PropertyName', 'SimilarityScore', 'Distance (km)'])

        return recommendations_df

    except KeyError:
        print(f"Error: '{property_name}' not found in location data.")
        return None

# User Interaction (CLI-based)
def main():
    print("\n==== Select Location and Radius ====")
    selected_location = input("Enter a location from the list:\n" + ", ".join(sorted(location_df.columns.to_list())) + "\n")

    try:
        radius = float(input("Enter radius in kilometers: "))

        # **Filter apartments within the given radius**
        filtered_df = location_df[location_df[selected_location] < radius * 1000][[selected_location]]
        filtered_df = filtered_df.sort_values(by=selected_location)

        print("\nProperties within", radius, "km:")
        for key, value in filtered_df[selected_location].items():
            print(f"{key}: {round(value / 1000, 2)} kms")

        if filtered_df.empty:
            print("\nNo properties found within the given radius. Try a larger radius.")
            return

        # **Now, let the user select from the filtered apartments**
        print("\n==== Recommend Apartments ====")
        selected_apartment = input("Enter an apartment name from the filtered list:\n" + ", ".join(filtered_df.index.to_list()) + "\n")

        if selected_apartment not in filtered_df.index:
            print("Invalid apartment selection. Please select from the filtered list.")
            return

        # **Pass only the filtered apartments to the recommendation function**
        recommendation_df = recommend_properties_with_scores(selected_apartment, filtered_df)

        if recommendation_df is not None and not recommendation_df.empty:
            print("\nRecommended Apartments:")
            print(recommendation_df.to_string(index=False))
        else:
            print("No similar apartments found within the selected radius.")

    except KeyError:
        print("Invalid location selected.")
    except ValueError:
        print("Invalid input. Please enter a number for radius.")

if __name__ == "__main__":
    main()



==== Select Location and Radius ====

Properties within 3.0 km:

No properties found within the given radius. Try a larger radius.


In [81]:
location_df['AIIMS']

PropertyName
Smartworld One DXP             54000.0
M3M Crown                      54000.0
Adani Brahma Samsara Vilasa    54000.0
Sobha City                     54000.0
Signature Global City 93       54000.0
                                ...   
DLF Princeton Estate           54000.0
Pyramid Urban Homes 2          54000.0
Satya The Hermitage            54000.0
BPTP Spacio                    54000.0
SS The Coralwood               54000.0
Name: AIIMS, Length: 246, dtype: float64